In [19]:
import csv
import json
import pandas as pd
from datetime import datetime
import numpy as np
from os.path import exists

from collections import Counter
import warnings
warnings.filterwarnings("ignore")


In [20]:
outputFolders = "../data_2019_processed/form/"

baseInputFolder = "../../Dataset_ECUADOR_2019/"

# Dataframe of all datasets
# initialize list of lists
datasets = [['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs'],
            ['0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA'], 
            ['2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0'], 
            ['2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys'], 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            ['7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA'], 
            ['a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4'], 
            ['ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc'], 
            ['Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U'], 
            ['CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA'], 
            ['DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc'], 
            ['DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA'], 
            ['dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY'], 
            ['HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo'], 
            ['jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw'], 
            ['JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I'], 
            ['K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8'], 
            ['oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q'], 
            ['pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM'], 
            ['PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc'], 
            ['QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ'], 
            ['rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw'], 
            ['RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI'], 
            ['SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'], 
            ['VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is'], 
            ['Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw'], 
            ['XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA'], 
            ['YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw'], 
            ['ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM'], 
            ['ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']]
  
# Create the pandas DataFrame
folders = pd.DataFrame(datasets, columns=['folder'])


In [21]:
#pd.read_csv("../Dataset_ECUADOR_2019/oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q/sleep_form_oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q.csv") # testFile = "../Dataset_ECUADOR_2019/oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q/sleep_form_oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q.csv"

In [22]:
def isSameDayConflict(timestampA,timestampB): # limit 
    #2018-06-02T15:21:58Z
    onlyDateA =  timestampA.split("T")
    onlyDateB =  timestampB.split("T")
    tempA = datetime.strptime(onlyDateA[0],'%Y-%m-%d').timestamp()
    tempB = datetime.strptime(onlyDateB[0],'%Y-%m-%d').timestamp()
    if(tempA==tempB):
        return True
    else:
        return False

def mapDuplicatedReports(dataframe):
    # create duplicated column flag, if it not exists
    #if not ('duplicated' in dataframe.columns):
    dataframe['duplicated'] = 0
    # limit must be -1 because we compare with the next value
    limit = dataframe.shape[0] - 1
    # reset index
    dataframe = dataframe.reset_index()
    # do the analysis
    for index in range(0,limit):
        current = dataframe['wakeup_time'][index]
        #print(current)
        if current >= dataframe['sleep_time'][index+1]:
            #print(index,dataframe['sleep_time'][index],dataframe['time_sleeping_s'][index],dataframe['wakeup_time'][index],dataframe['timestamp'][index])
            #print(dataframe['timestamp'][index],dataframe['timestamp'][index+1])
            if isSameDayConflict(dataframe['timestamp'][index],dataframe['timestamp'][index+1]):
                dataframe['duplicated'][index] = 1
                dataframe['duplicated'][index+1] = 0
    return dataframe
                
print("Conflict",isSameDayConflict('2018-06-02T15:21:58Z','2018-06-02T15:21:58Z'))
print("Not conflict",isSameDayConflict('2018-06-02T15:21:58Z','2018-06-03T15:21:58Z'))

Conflict True
Not conflict False


In [23]:
print("Evaluation: script to deal with negative sleep times")
test = []
temposDuplicadas = 0
temposMaioresDe12 = 0
temposMenoresDe2 = 0
temposNegativos = 0
temposTotal = 0
temposUseful = 0


countIndex = 0
for indexFolder, folder in folders.iterrows():
    #testFile = "2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys.csv"
    
    # check f the path exists (becouse kaggle removed the + character from file names)
    formFileName = "sleep_form_"+folder['folder']+".csv"
    if(not(exists(baseInputFolder+folder['folder']+"/"+formFileName))):
        formFileName = "sleep_form_"+folder['folder'].replace("+","")+".csv"
        
    print(formFileName)
    file = baseInputFolder+folder['folder']+"/"+formFileName
    dataset = pd.read_csv(file)
    data = dict()
    
    # test transformation
    dataframe = dataset
    # sleep_time to timestamp
    for indexFolder, content in dataframe['sleep_time'].items():
        temp = datetime.strptime(content,'%Y-%m-%dT%H:%M:%SZ')
        dataframe['sleep_time'][indexFolder] = temp.timestamp()

    # wakeup_time to timestamp
    for indexFolder, content in dataframe['wakeup_time'].items():
        temp = datetime.strptime(content,'%Y-%m-%dT%H:%M:%SZ')
        dataframe['wakeup_time'][indexFolder] = temp.timestamp() 
        
        # verify if the wakeup_time and sleep_time are inverted
        temp_time = dataframe['sleep_time'][indexFolder]
        if(temp_time > dataframe['wakeup_time'][indexFolder]):
            dataframe['sleep_time'][indexFolder]  = dataframe['wakeup_time'][indexFolder]
            dataframe['wakeup_time'][indexFolder] = temp_time

    # time sleeling in seconds
    dataframe['time_sleeping'] = dataframe['wakeup_time'] - dataframe['sleep_time']
    # convert time sleeping to hours
    dataframe['time_sleeping_hs'] = dataframe['time_sleeping'] /60/60
            
    # replace in 24hs of max sleeping
    #for indexFolder, content in dataframe['time_sleeping_hs'].items():
    #    if(content > 24.0):
    #        dataframe['time_sleeping'][indexFolder] = 24.0 *60 *60
    #        dataframe['time_sleeping_hs'][indexFolder] = 24.0
    #        dataframe['wakeup_time'][indexFolder] = dataframe['sleep_time'][indexFolder] + dataframe['time_sleeping'][indexFolder]
    
    # remove forms with more than then 12hs
    dfOnlyWithLess12hs = dataframe.loc[dataframe['time_sleeping_hs'] <= 24.0]
    dfmorethan2hs = dfOnlyWithLess12hs.loc[dataframe['time_sleeping_hs'] >= 2.0]
    # map forms with same day of report
    dfOnlyUnique = mapDuplicatedReports(dfmorethan2hs)
    # remove duplicates
    dfOnlyUnique = dfOnlyUnique.loc[dfOnlyUnique['duplicated'] == 0]
    # remove forms with zero hours of sleep
    dfOnlyUseful = dfOnlyUnique.loc[dfOnlyUnique['time_sleeping_hs'] > 0.0]
   
    temposTotal = temposTotal + dataframe.shape[0]
    temposMaioresDe12 = temposMaioresDe12 + dataframe.shape[0]-dfOnlyWithLess12hs.shape[0]
    temposMenoresDe2 = temposMenoresDe2 + (dfOnlyWithLess12hs.shape[0]-dfmorethan2hs.shape[0])
    temposDuplicadas = temposDuplicadas + (dfmorethan2hs.shape[0]-dfOnlyUnique.shape[0])
    temposNegativos = temposNegativos + dfOnlyUnique.loc[dfOnlyUnique['time_sleeping_hs'] <= 0.0].shape[0] 
    temposUseful = temposUseful + dfOnlyUseful.shape[0]
        
    print("")
    print("")
    print("")
    print("Client:",countIndex,folder['folder'])
    print("Total count: ",dataframe.shape[0])    
    print("Count with less 24hs: ",dfOnlyWithLess12hs.shape[0], " - removed: ",dataframe.shape[0]-dfOnlyWithLess12hs.shape[0])
    print("Count with more than 2hs: ",dfmorethan2hs.shape[0], " - removed: ",dfOnlyWithLess12hs.shape[0]-dfmorethan2hs.shape[0])
    print("Count without duplicates: ",dfOnlyUnique.shape[0]," - removed: ",dfmorethan2hs.shape[0]-dfOnlyUnique.shape[0])
    print("Count negatives: ",dfOnlyUnique.loc[dfOnlyUnique['time_sleeping_hs'] <= 0.0].shape[0])
    print("Count useful: ",dfOnlyUseful.shape[0])
    #print([dataframe['time_sleeping'],dataframe['time_sleeping']])
    #test.append(data)
    countIndex = countIndex + 1
        
    #countSleep = 0
    #for indexData in range(0,len(dataDF)):
    #    rowData = dataDF['awake'][indexData]
    
temposNormais = temposTotal - temposDuplicadas - temposMaioresDe12 - temposNegativos  - temposMenoresDe2
print("")
print("")
print("")
print("Total:",temposTotal)    
print("duplicated:",temposDuplicadas)
print(">24:",temposMaioresDe12)
print("0-24:",temposNormais, "useful")
print("<=2:",temposMenoresDe2)
print("<=0:",temposNegativos)
print("Useful:",temposUseful, "useful")

Evaluation: script to deal with negative sleep times
sleep_form_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs.csv



Client: 0 0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs
Total count:  33
Count with less 24hs:  33  - removed:  0
Count with more than 2hs:  32  - removed:  1
Count without duplicates:  26  - removed:  6
Count negatives:  0
Count useful:  26
sleep_form_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA.csv



Client: 1 0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA
Total count:  31
Count with less 24hs:  31  - removed:  0
Count with more than 2hs:  31  - removed:  0
Count without duplicates:  22  - removed:  9
Count negatives:  0
Count useful:  22
sleep_form_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0.csv



Client: 2 2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0
Total count:  48
Count with less 24hs:  48  - removed:  0
Count with more than 2hs:  46  - removed:  2
Count without duplicates:  29  - removed:  17
Count negatives:  0
Count useful:  29
sleep_form_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6

In [25]:
print("Evaluation: script to deal with negative sleep times")
test = []
temposDuplicadas = 0
temposMaioresDe12 = 0
temposMenoresDe2 = 0
temposNegativos = 0
temposTotal = 0
temposUseful = 0


countIndex = 0
for indexFolder, folder in folders.iterrows():
    #testFile = "2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys.csv"
    
    # check f the path exists (becouse kaggle removed the + character from file names)
    formFileName = "sleep_form_"+folder['folder']+".csv"
    if(not(exists(baseInputFolder+folder['folder']+"/"+formFileName))):
        formFileName = "sleep_form_"+folder['folder'].replace("+","")+".csv"
        
    print(formFileName)
    file = baseInputFolder+folder['folder']+"/"+formFileName
    dataset = pd.read_csv(file)
    data = dict()
    
    # test transformation
    dataframe = dataset
    # sleep_time to timestamp
    for indexFolder, content in dataframe['sleep_time'].items():
        temp = datetime.strptime(content,'%Y-%m-%dT%H:%M:%SZ')
        dataframe['sleep_time'][indexFolder] = temp.timestamp()

    # wakeup_time to timestamp
    for indexFolder, content in dataframe['wakeup_time'].items():
        temp = datetime.strptime(content,'%Y-%m-%dT%H:%M:%SZ')
        dataframe['wakeup_time'][indexFolder] = temp.timestamp() 
        
        # verify if the wakeup_time and sleep_time are inverted
        temp_time = dataframe['sleep_time'][indexFolder]
        if(temp_time > dataframe['wakeup_time'][indexFolder]):
            dataframe['sleep_time'][indexFolder]  = dataframe['wakeup_time'][indexFolder]
            dataframe['wakeup_time'][indexFolder] = temp_time

    # time sleeling in seconds
    dataframe['time_sleeping'] = dataframe['wakeup_time'] - dataframe['sleep_time']
    # convert time sleeping to hours
    dataframe['time_sleeping_hs'] = dataframe['time_sleeping'] /60/60
            
    # replace in 24hs of max sleeping
    #for indexFolder, content in dataframe['time_sleeping_hs'].items():
    #    if(content > 24.0):
    #        dataframe['time_sleeping'][indexFolder] = 24.0 *60 *60
    #        dataframe['time_sleeping_hs'][indexFolder] = 24.0
    #        dataframe['wakeup_time'][indexFolder] = dataframe['sleep_time'][indexFolder] + dataframe['time_sleeping'][indexFolder]
    
    # remove forms with more than then 12hs
    dfOnlyWithLess12hs = dataframe.loc[dataframe['time_sleeping_hs'] <= 24.0]
    dfmorethan2hs = dfOnlyWithLess12hs.loc[dataframe['time_sleeping_hs'] >= 2.0]
    # map forms with same day of report
    dfOnlyUnique = mapDuplicatedReports(dfmorethan2hs)
    # remove duplicates
    dfOnlyUnique = dfOnlyUnique.loc[dfOnlyUnique['duplicated'] == 0]
    # remove forms with zero hours of sleep
    dfOnlyUseful = dfOnlyUnique.loc[dfOnlyUnique['time_sleeping_hs'] > 0.0]
   
    temposTotal = temposTotal + dataframe.shape[0]
    temposMaioresDe12 = temposMaioresDe12 + dataframe.shape[0]-dfOnlyWithLess12hs.shape[0]
    temposMenoresDe2 = temposMenoresDe2 + (dfOnlyWithLess12hs.shape[0]-dfmorethan2hs.shape[0])
    temposDuplicadas = temposDuplicadas + (dfmorethan2hs.shape[0]-dfOnlyUnique.shape[0])
    temposNegativos = temposNegativos + dfOnlyUnique.loc[dfOnlyUnique['time_sleeping_hs'] <= 0.0].shape[0] 
    temposUseful = temposUseful + dfOnlyUseful.shape[0]
    
    # write in file
    fileOutput = outputFolders+"sleep_form_"+folder['folder']+"_treated.csv"
    dfOnlyUseful.to_csv(fileOutput)
        
    print("")
    print("")
    print("")
    print("Client:",countIndex,folder['folder'])
    print("Total count: ",dataframe.shape[0])    
    print("Count with less 24hs: ",dfOnlyWithLess12hs.shape[0], " - removed: ",dataframe.shape[0]-dfOnlyWithLess12hs.shape[0])
    print("Count with more than 2hs: ",dfmorethan2hs.shape[0], " - removed: ",dfOnlyWithLess12hs.shape[0]-dfmorethan2hs.shape[0])
    print("Count without duplicates: ",dfOnlyUnique.shape[0]," - removed: ",dfmorethan2hs.shape[0]-dfOnlyUnique.shape[0])
    print("Count negatives: ",dfOnlyUnique.loc[dfOnlyUnique['time_sleeping_hs'] <= 0.0].shape[0])
    print("Count useful: ",dfOnlyUseful.shape[0])
    #print([dataframe['time_sleeping'],dataframe['time_sleeping']])
    #test.append(data)
    countIndex = countIndex + 1
        
    #countSleep = 0
    #for indexData in range(0,len(dataDF)):
    #    rowData = dataDF['awake'][indexData]
    
temposNormais = temposTotal - temposDuplicadas - temposMaioresDe12 - temposNegativos - temposMenoresDe2
print("")
print("")
print("")
print("Total:",temposTotal)    
print("duplicated:",temposDuplicadas)
print(">24:",temposMaioresDe12)
print("0-24:",temposNormais, "useful")
print("<=2:",temposMenoresDe2)
print("<=0:",temposNegativos)
print("Useful:",temposUseful, "useful")

Evaluation: script to deal with negative sleep times
sleep_form_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs.csv



Client: 0 0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs
Total count:  33
Count with less 24hs:  33  - removed:  0
Count with more than 2hs:  32  - removed:  1
Count without duplicates:  26  - removed:  6
Count negatives:  0
Count useful:  26
sleep_form_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA.csv



Client: 1 0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA
Total count:  31
Count with less 24hs:  31  - removed:  0
Count with more than 2hs:  31  - removed:  0
Count without duplicates:  22  - removed:  9
Count negatives:  0
Count useful:  22
sleep_form_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0.csv



Client: 2 2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0
Total count:  48
Count with less 24hs:  48  - removed:  0
Count with more than 2hs:  46  - removed:  2
Count without duplicates:  29  - removed:  17
Count negatives:  0
Count useful:  29
sleep_form_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6




Client: 28 ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY
Total count:  21
Count with less 24hs:  20  - removed:  1
Count with more than 2hs:  17  - removed:  3
Count without duplicates:  17  - removed:  0
Count negatives:  0
Count useful:  17



Total: 965
duplicated: 278
>24: 74
0-24: 593 useful
<=2: 20
<=0: 0
Useful: 593 useful


In [26]:
dfOnlyUseful

,index,Unnamed: 0,sleep_time,wakeup_time,sleep,arousal,valence,dominance,physical_activity,sociability,study,timestamp,time_sleeping,time_sleeping_hs,duplicated
0,0,0,1526591743.0,1526608843.0,1,2,4,1,1,4,2,2018-05-18T03:22:43Z,17100.0,4.75,0
1,1,1,1526613300.0,1526642112.0,2,0,1,3,0,0,0,2018-05-18T12:21:12Z,28812.0,8.003333,0
2,2,2,1526623210.0,1526634010.0,2,2,3,3,0,4,3,2018-05-19T03:48:10Z,10800.0,3.0,0
3,3,3,1526794252.0,1526823052.0,0,0,0,0,0,1,2,2018-05-20T14:39:52Z,28800.0,8.0,0
4,4,4,1526796000.0,1526875236.0,2,4,4,1,1,2,1,2018-05-21T05:50:36Z,79236.0,22.01,0
5,5,5,1526884224.0,1526902224.0,4,1,4,2,2,3,3,2018-05-21T15:40:24Z,18000.0,5.0,0
6,6,6,1526962530.0,1527005730.0,4,4,2,2,3,4,4,2018-05-22T05:16:30Z,43200.0,12.0,0
7,9,9,1527314410.0,1527350410.0,1,1,4,3,3,4,1,2018-05-26T17:08:10Z,36000.0,10.0,0
8,10,10,1527400832.0,1527418832.0,4,4,1,2,3,4,0,2018-05-27T15:13:32Z,18000.0,5.0,0
9,11,11,1527487213.0,1527505213.0,4,4,4,4,3,4,2,2018-05-28T14:54:13Z,18000.0,5.0,0
